# LFT: Gravity from Strain Geometry (Proof of Concept)

## Objective
Derive gravitational phenomena from persistent logical strain on the permutohedron, using the strain tensor formalism from LFT_03.5:
- Matter/Energy = localized regions of high persistent strain
- Gravity = modification of strain flow paths due to strain accumulation
- Metric = effective distance on permutohedron modified by strain density

This notebook:
1. Works with actual permutohedral geometry (not proxy lattices)
2. Derives the strain potential from first principles
3. Shows how persistent strain modifies the graph metric
4. Demonstrates time dilation and geodesic bending
5. Connects to field equations in continuum limit

## 1. Setup: Strain on the Permutohedron

From LFT_03.5, the strain between positions is:
$$s_{ij}(\\sigma) = \\text{sign}(\\sigma(i) - \\sigma(j))$$

Total strain (inversion count):
$$h(\\sigma) = \\sum_{i<j} \\frac{1}{2}(1 + s_{ij}(\\sigma))$$

The permutohedron Pi_{N-1} has:
- Vertices: permutations sigma in S_N
- Edges: adjacent transpositions (sigma, sigma s_i)
- Natural metric: graph distance

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from itertools import permutations
from collections import defaultdict
import heapq

def inversion_count(perm):
    """Count inversions in a permutation"""
    h = 0
    n = len(perm)
    for i in range(n):
        for j in range(i+1, n):
            if perm[i] > perm[j]:
                h += 1
    return h

def adjacent_swaps(perm):
    """Generate all adjacent swap neighbors"""
    n = len(perm)
    neighbors = []
    for i in range(n-1):
        neighbor = list(perm)
        neighbor[i], neighbor[i+1] = neighbor[i+1], neighbor[i]
        neighbors.append(tuple(neighbor))
    return neighbors

def build_permutohedron_graph(n):
    """Build the permutohedron graph for S_n"""
    G = nx.Graph()
    perms = list(permutations(range(n)))
    
    # Add vertices with strain as attribute
    for perm in perms:
        h = inversion_count(perm)
        G.add_node(perm, strain=h)
    
    # Add edges
    for perm in perms:
        for neighbor in adjacent_swaps(perm):
            if neighbor in G:
                G.add_edge(perm, neighbor)
    
    return G

# Build permutohedron for N=4 (24 vertices)
G4 = build_permutohedron_graph(4)
print(f"Permutohedron S_4: {G4.number_of_nodes()} vertices, {G4.number_of_edges()} edges")

## 2. Persistent Strain as Matter

Key insight: Matter corresponds to regions where logical strain cannot easily dissipate - "knots" in the logical structure.

We model this by introducing strain sources that maintain elevated strain in their vicinity:

$$\\rho(\\sigma) = \\sum_k M_k \\cdot K(d(\\sigma, \\sigma_k))$$

where:
- sigma_k are source locations
- M_k is the source strength ("mass")
- K is a kernel function (strain propagator)
- d is graph distance on the permutohedron

In [ ]:
def graph_distance_matrix(G):
    """Compute all-pairs shortest path distances"""
    nodes = list(G.nodes())
    n = len(nodes)
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    
    # Use Floyd-Warshall for small graphs
    dist = np.full((n, n), np.inf)
    np.fill_diagonal(dist, 0)
    
    for u, v in G.edges():
        i, j = node_to_idx[u], node_to_idx[v]
        dist[i, j] = dist[j, i] = 1
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                dist[i, j] = min(dist[i, j], dist[i, k] + dist[k, j])
    
    return dist, nodes, node_to_idx

def strain_kernel(d, scale=2.0):
    """Kernel for strain propagation"""
    return np.exp(-d**2 / (2 * scale**2))

def compute_strain_density(G, sources, masses, scale=2.0):
    """Compute strain density field from sources"""
    dist_matrix, nodes, node_to_idx = graph_distance_matrix(G)
    n = len(nodes)
    density = np.zeros(n)
    
    for source, mass in zip(sources, masses):
        if source in node_to_idx:
            source_idx = node_to_idx[source]
            for i in range(n):
                d = dist_matrix[source_idx, i]
                density[i] += mass * strain_kernel(d, scale)
    
    return density, nodes

# Place a "mass" at a high-strain configuration
sources = [(2, 3, 0, 1)]  # A specific permutation
masses = [5.0]

density, nodes = compute_strain_density(G4, sources, masses)
print(f"Strain density range: [{density.min():.3f}, {density.max():.3f}]")

## 3. Modified Metric from Strain

High strain density makes logical transitions more "costly". The effective metric becomes:

$$d_{\\text{eff}}(\\sigma_1, \\sigma_2) = \\int_{\\gamma} \\sqrt{1 + \\alpha \\rho(\\sigma)} \\, ds$$

where gamma is the path from sigma_1 to sigma_2.

For discrete paths on the permutohedron:
$$d_{\\text{eff}} = \\sum_{\\text{edges}} \\frac{1}{2}[\\sqrt{1 + \\alpha \\rho(\\sigma_i)} + \\sqrt{1 + \\alpha \\rho(\\sigma_{i+1})}]$$

In [ ]:
def edge_weight_from_strain(G, node1, node2, density, node_to_idx, alpha=1.0):
    """Compute edge weight based on strain density"""
    idx1 = node_to_idx[node1]
    idx2 = node_to_idx[node2]
    
    # Average the metric factor at both endpoints
    factor1 = np.sqrt(1 + alpha * density[idx1])
    factor2 = np.sqrt(1 + alpha * density[idx2])
    
    return 0.5 * (factor1 + factor2)

def build_weighted_graph(G, density, nodes, alpha=1.0):
    """Create weighted graph with strain-modified metric"""
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    G_weighted = nx.Graph()
    
    # Add all nodes
    for i, node in enumerate(nodes):
        G_weighted.add_node(node, strain=G.nodes[node]['strain'], 
                          density=density[i])
    
    # Add weighted edges
    for u, v in G.edges():
        weight = edge_weight_from_strain(G, u, v, density, node_to_idx, alpha)
        G_weighted.add_edge(u, v, weight=weight)
    
    return G_weighted

# Create weighted graph
alpha = 2.0  # Coupling strength
G_weighted = build_weighted_graph(G4, density, nodes, alpha)

# Compare distances
source_node = (0, 1, 2, 3)
target_node = (3, 2, 1, 0)

# Unweighted distance
dist_unweighted = nx.shortest_path_length(G4, source_node, target_node)

# Weighted distance
dist_weighted = nx.shortest_path_length(G_weighted, source_node, target_node, weight='weight')

print(f"Unweighted distance: {dist_unweighted}")
print(f"Weighted distance: {dist_weighted:.3f}")
print(f"Metric distortion factor: {dist_weighted/dist_unweighted:.3f}")

## 4. Time Dilation from Strain Flow

From LFT_03.5, the rate of strain relief (and thus time flow) is:

$$\\frac{dh}{dt} = -\\kappa(\\sigma) |\\nabla h|$$

In regions of high strain density, the effective kappa is reduced:

$$\\kappa_{\\text{eff}}(\\sigma) = \\frac{\\kappa_0}{1 + \\beta \\rho(\\sigma)}$$

This gives gravitational time dilation: clocks run slower in regions of high strain density.

In [ ]:
def compute_time_dilation(density, beta=1.0):
    """Compute time dilation factor from strain density"""
    return 1.0 / (1.0 + beta * density)

def simulate_strain_flow(G_weighted, initial_node, steps=20, beta=1.0):
    """Simulate strain flow with time dilation"""
    node_to_idx = {node: i for i, node in enumerate(G_weighted.nodes())}
    density_array = np.array([G_weighted.nodes[node]['density'] 
                             for node in G_weighted.nodes()])
    
    current = initial_node
    trajectory = [current]
    proper_times = [0.0]
    coordinate_time = 0
    
    for step in range(steps):
        # Get neighbors that reduce strain
        neighbors = list(G_weighted.neighbors(current))
        current_strain = inversion_count(current)
        
        downhill = [(n, inversion_count(n)) for n in neighbors 
                   if inversion_count(n) < current_strain]
        
        if not downhill:
            break
        
        # Choose steepest descent
        next_node = min(downhill, key=lambda x: x[1])[0]
        
        # Time dilation at current location
        current_idx = node_to_idx[current]
        dilation = compute_time_dilation(density_array[current_idx], beta)
        
        # Update times
        coordinate_time += 1
        proper_times.append(proper_times[-1] + dilation)
        
        current = next_node
        trajectory.append(current)
    
    return trajectory, proper_times, coordinate_time

# Compare time flow near and far from mass
near_mass = sources[0]  # Start at the mass
far_from_mass = (0, 1, 2, 3)  # Identity permutation

traj_near, proper_near, coord_near = simulate_strain_flow(G_weighted, near_mass, beta=1.0)
traj_far, proper_far, coord_far = simulate_strain_flow(G_weighted, far_from_mass, beta=1.0)

if len(proper_near) > 1 and len(proper_far) > 1:
    print(f"Near mass: {coord_near} coordinate steps, {proper_near[-1]:.3f} proper time")
    print(f"Far from mass: {coord_far} coordinate steps, {proper_far[-1]:.3f} proper time")
    if coord_near > 0:
        print(f"Time dilation factor: {proper_near[-1]/coord_near:.3f}")

## 5. Geodesics and Strain Lensing

Shortest paths in the strain-modified metric bend around regions of high strain density, analogous to gravitational lensing.

In [ ]:
def visualize_geodesics(G_weighted, source, target, mass_location):
    """Compare geodesics with and without strain"""
    # Unweighted shortest path
    path_unweighted = nx.shortest_path(G4, source, target)
    
    # Weighted shortest path (geodesic)
    path_weighted = nx.shortest_path(G_weighted, source, target, weight='weight')
    
    # Convert paths to strain values for visualization
    strain_unweighted = [inversion_count(node) for node in path_unweighted]
    strain_weighted = [inversion_count(node) for node in path_weighted]
    
    # Check if paths differ
    paths_differ = (path_unweighted != path_weighted)
    
    print(f"\nGeodesic analysis:")
    print(f"Unweighted path length: {len(path_unweighted)-1} steps")
    print(f"Weighted path length: {len(path_weighted)-1} steps")
    print(f"Paths differ: {paths_differ}")
    
    if paths_differ:
        print(f"\nPath bending detected!")
        print(f"Unweighted path passes through strains: {strain_unweighted}")
        print(f"Weighted path passes through strains: {strain_weighted}")
    
    return path_unweighted, path_weighted

# Test geodesics
source = (0, 1, 2, 3)
target = (2, 3, 1, 0)
path_straight, path_curved = visualize_geodesics(G_weighted, source, target, sources[0])

## 6. Effective Field Equations

In the continuum limit, the strain density rho satisfies a field equation. From the discrete Laplacian on the permutohedron:

$$\\Delta \\rho = -4\\pi G \\rho_m$$

where:
- Delta is the graph Laplacian
- rho_m is the matter density (persistent strain sources)
- G emerges from the coupling alpha

This is the discrete analog of Poisson's equation for gravity.

In [ ]:
def compute_graph_laplacian(G_weighted, density, nodes):
    """Compute discrete Laplacian of density field"""
    n = len(nodes)
    laplacian = np.zeros(n)
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    
    for i, node in enumerate(nodes):
        degree = G_weighted.degree(node, weight='weight')
        
        # Laplacian: sum of differences with neighbors
        for neighbor in G_weighted.neighbors(node):
            j = node_to_idx[neighbor]
            weight = G_weighted[node][neighbor]['weight']
            laplacian[i] += (density[j] - density[i]) / weight
    
    return laplacian

def verify_field_equation(G_weighted, density, nodes, sources, masses):
    """Check if density satisfies discrete field equation"""
    # Compute Laplacian
    laplacian = compute_graph_laplacian(G_weighted, density, nodes)
    
    # Compute source term
    source_term = np.zeros(len(nodes))
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    
    for source, mass in zip(sources, masses):
        if source in node_to_idx:
            idx = node_to_idx[source]
            source_term[idx] = mass
    
    # Check correlation
    correlation = np.corrcoef(laplacian[source_term > 0], 
                             source_term[source_term > 0])[0, 1]
    
    print(f"\nField equation verification:")
    print(f"Max |Laplacian|: {np.abs(laplacian).max():.3f}")
    print(f"Laplacian at source: {laplacian[node_to_idx[sources[0]]]:.3f}")
    print(f"Correlation with source: {correlation:.3f}")
    
    return laplacian, source_term

laplacian, source_term = verify_field_equation(G_weighted, density, nodes, sources, masses)

## 7. Emergence of Continuous Spacetime

For large N, the permutohedron becomes quasi-continuous. The effective dimension is N-1, and our 3+1D spacetime emerges at N=4 (from computational feasibility arguments).

The coupling constants relate as:
$$G \\sim \\frac{\\alpha}{N^2} \\cdot l_P^2$$

where l_P is the Planck length (minimum logical distinction scale).

In [ ]:
# Analyze scaling with N
def analyze_scaling():
    """Study how strain effects scale with N"""
    results = []
    
    for n in [3, 4, 5]:  # Limited by computation
        G_n = build_permutohedron_graph(n)
        
        # Average edge density
        avg_degree = 2 * G_n.number_of_edges() / G_n.number_of_nodes()
        
        # Diameter
        diameter = nx.diameter(G_n)
        
        # Strain statistics
        strains = [inversion_count(node) for node in G_n.nodes()]
        max_strain = max(strains)
        
        results.append({
            'N': n,
            'vertices': G_n.number_of_nodes(),
            'edges': G_n.number_of_edges(),
            'avg_degree': avg_degree,
            'diameter': diameter,
            'max_strain': max_strain
        })
    
    return results

scaling_results = analyze_scaling()
print("\nScaling with N:")
for res in scaling_results:
    print(f"N={res['N']}: {res['vertices']} vertices, "
          f"diameter={res['diameter']}, max_strain={res['max_strain']}")

# Effective gravitational constant
def effective_G(N, alpha, l_planck=1.0):
    """Estimate effective gravitational constant"""
    return alpha * l_planck**2 / N**2

print("\nEffective G:")
for n in [3, 4, 5, 6]:
    print(f"N={n}: G_eff = {effective_G(n, alpha):.3f}")

## 8. Summary and Visualizations

In [ ]:
# Create summary visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Strain density distribution
ax1 = axes[0, 0]
strain_values = [inversion_count(node) for node in nodes]
scatter = ax1.scatter(strain_values, density, c=density, cmap='Reds', s=50)
ax1.set_xlabel('Intrinsic strain h(sigma)')
ax1.set_ylabel('Strain density rho(sigma)')
ax1.set_title('Strain Density vs Intrinsic Strain')
plt.colorbar(scatter, ax=ax1)

# 2. Time dilation map
ax2 = axes[0, 1]
time_dilation = compute_time_dilation(density)
ax2.scatter(strain_values, time_dilation, c=density, cmap='Blues', s=50)
ax2.set_xlabel('Intrinsic strain h(sigma)')
ax2.set_ylabel('Time dilation factor')
ax2.set_title('Gravitational Time Dilation')

# 3. Degree distribution with strain
ax3 = axes[1, 0]
degrees = [G_weighted.degree(node, weight='weight') for node in nodes]
ax3.scatter(strain_values, degrees, c=density, cmap='Greens', s=50)
ax3.set_xlabel('Intrinsic strain h(sigma)')
ax3.set_ylabel('Weighted degree')
ax3.set_title('Effective Connectivity vs Strain')

# 4. Field equation residual
ax4 = axes[1, 1]
ax4.scatter(source_term, -laplacian, s=50)
ax4.set_xlabel('Source term rho_m')
ax4.set_ylabel('-Delta rho')
ax4.set_title('Field Equation: -Delta rho vs rho_m')
ax4.axline((0, 0), slope=1, color='red', linestyle='--', label='Linear relation')
ax4.legend()

plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_strain_gravity_analysis.png', dpi=200)
plt.show()

# Summary statistics
print("\nSummary:")
print(f"Strain density range: [{density.min():.3f}, {density.max():.3f}]")
print(f"Time dilation range: [{time_dilation.min():.3f}, {time_dilation.max():.3f}]")
print(f"Maximum metric distortion: {max(degrees)/min(degrees):.3f}")
print(f"Effective dimension: {3} (for N=4)")

## 9. Conclusions

This notebook demonstrates that gravitational phenomena emerge naturally from persistent logical strain on the permutohedron:

1. **Matter = Persistent strain**: Regions where logical constraints cannot easily resolve
2. **Curved metric**: High strain density increases the "cost" of logical transitions
3. **Time dilation**: Strain relief (and thus time flow) slows in high-density regions
4. **Geodesic deviation**: Optimal paths bend around strain concentrations
5. **Field equations**: Discrete Poisson equation emerges for strain density

Key insights:
- Gravity is not a force but the geometry of constraint resolution
- The coupling alpha determines the strength of gravitational effects
- For N=4, we get 3D space with emergent gravitational phenomena
- The discrete structure naturally regularizes singularities

Next steps:
- Derive Einstein equations in continuum limit
- Study dynamic strain source formation
- Connect to quantum effects (strain superposition)
- Investigate cosmological solutions (homogeneous strain)